In [1]:
import AdventOfCode

In [2]:
input <- dayLines 17

In [3]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Attoparsec.ByteString.Char8 as C

parseXY = do
    C.string "x="
    x <- C.decimal
    C.string ", y="
    yl <- C.decimal
    C.string ".."
    yr <- C.decimal
    pure [(x,y) | y <- [yl..yr]]

parseYX = do
    C.string "y="
    y <- C.decimal
    C.string ", x="
    xl <- C.decimal
    C.string ".."
    xr <- C.decimal
    pure [(x,y) | x <- [xl..xr]]

import Control.Applicative

parseLine = parseXY <|> parseYX

points = concatMap (parsed parseLine) input

In [4]:
import Data.Function (on)
import Data.List

maxX = fst $ maximumBy (compare `on` fst) points
maxY = snd $ maximumBy (compare `on` snd) points


maxX
maxY
length points

594

1801

23894

In [13]:
minX = fst $ minimumBy (compare `on` fst) points
minY = snd $ minimumBy (compare `on` snd) points
minX
minY

314

3

In [6]:
import qualified Data.Set as S
import qualified Data.Vector as V

pointSet = S.fromList points

row = V.replicate (maxX+1) ' '
tiles = V.replicate (maxY+1) row

rowSet y rowLen pSet = [(x, '#') | x <- [0..(rowLen-1)], S.member (x,y) pSet]

In [7]:
rowSet 1801 594 pointSet

process grid pSet n | n >= V.length grid = grid
process grid pSet n = let
   pts = rowSet n (V.length (V.head grid)) pSet
   row' = (grid V.! n) V.// pts
   grid' = grid V.// [(n, row')]
   in process grid' pSet (n+1)

[(314,'#'),(315,'#'),(316,'#'),(317,'#'),(318,'#'),(319,'#'),(320,'#'),(321,'#'),(322,'#'),(323,'#'),(324,'#'),(325,'#'),(326,'#'),(327,'#'),(328,'#'),(329,'#'),(330,'#')]

In [8]:
processed = process tiles pointSet 0

In [9]:
undergroundMap = unlines $ V.toList $ fmap V.toList processed

In [12]:
writeFile "map.txt" undergroundMap